## knock 084 分母が欠損値や0の場合でも除算結果データを作成する

In [24]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [25]:
df_customer = pl.read_csv("../docker/work/data/customer.csv")
display(df_customer.head())

df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071…","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂******…","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172…","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木*****…","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001…","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**…","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145…","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷*****…","""S13001""",20170605,"""6-20090929-2"""


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [26]:
(
    df_customer
    # 使用する列を抽出する
    .select([pl.col("customer_id")])
    # 2019年の売上高を追加する
    .join(
        # 2019年の顧客ごと売上高
        other = df_receipt
        # 整数型の2019年の日付に含まれている行を抽出する
        .filter(pl.col('sales_ymd').is_between(20190000, 20191231, closed = 'both'))
        .group_by("customer_id")
        .agg(pl.sum("amount")
        .alias("sales_amount_2019")),
        on = "customer_id",
        how = "left"
    )
    # 全期間の売上高を追加する
    .join(
        other = df_receipt
        .group_by("customer_id")
        .agg(pl.sum("amount")
        .alias("sales_amount_all")),
        on = "customer_id",
        how = "left"
    )
    # 2019年の売り上げ比率を計算する
    .with_columns(
        (pl.col("sales_amount_2019") / pl.col("sales_amount_all"))
        .alias("sales_rate")
    )
    # 0埋め
    .fill_null(0)
    # 売上比率が0より大きい顧客を抽出する
    .filter(
        pl.col('sales_rate') > 0
    )
    # 先頭10行を表示する
    .head(n = 10)
)

customer_id,sales_amount_2019,sales_amount_all,sales_rate
str,i64,i64,f64
"""CS031415000172…",2971,5088,0.583923
"""CS015414000103…",874,3122,0.279949
"""CS011215000048…",248,3444,0.072009
"""CS029415000023…",3767,5167,0.72905
"""CS035415000029…",5823,7504,0.775986
"""CS023513000066…",208,771,0.26978
"""CS035513000134…",463,1565,0.295847
"""CS001515000263…",216,216,1.0
"""CS006415000279…",229,229,1.0
